In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [ ]:
df=pd.read_csv('/content/youtoxic_english_1000.csv')

In [ ]:
df.head()

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
df.drop(columns=['CommentId','VideoId'],inplace=True)

In [ ]:
df.head()

,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False
3,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
df.shape

(1000, 13)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder=LabelEncoder()

In [ ]:
enci=['IsToxic'	,'IsAbusive',	'IsThreat',	'IsProvocative',	'IsObscene',	'IsHatespeech',	'IsRacist',	'IsNationalist',	'IsSexist',	'IsHomophobic',	'IsReligiousHate'	,'IsRadicalism']

In [ ]:
def encoding(df, enci):
  encoder = LabelEncoder()
  for col in enci:
    df[col] = encoder.fit_transform(df[col])
  return df

In [ ]:
df=encoding(df,enci)

In [ ]:
df.head()

,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,If only people would just take a step back and...,0,0,0,0,0,0,0,0,0,0,0,0
1,Law enforcement is not trained to shoot to app...,1,1,0,0,0,0,0,0,0,0,0,0
2,\nDont you reckon them 'black lives matter' ba...,1,1,0,0,1,0,0,0,0,0,0,0
3,There are a very large number of people who do...,0,0,0,0,0,0,0,0,0,0,0,0
4,"The Arab dude is absolutely right, he should h...",0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df['Text']

,Text
0,If only people would just take a step back and...
1,Law enforcement is not trained to shoot to app...
2,\nDont you reckon them 'black lives matter' ba...
3,There are a very large number of people who do...
4,"The Arab dude is absolutely right, he should h..."
...,...
995,I remember that they sent in the national defe...
996,Stats don`t represent the problem. Race baitin...
997,The quote from the mother... Wow that hit hard...
998,this video is so racist


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [ ]:
X=df['Text']
y=df.drop(columns=['Text'])

In [ ]:
X.head()

,Text
0,If only people would just take a step back and...
1,Law enforcement is not trained to shoot to app...
2,\nDont you reckon them 'black lives matter' ba...
3,There are a very large number of people who do...
4,"The Arab dude is absolutely right, he should h..."


In [ ]:
Max_features=200000

In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

In [ ]:
cleaned_X=X.map(clean_text)

In [ ]:
vectorizer=TextVectorization(max_tokens=Max_features,
                             output_sequence_length=1800,
                             output_mode='int')

In [ ]:
vectorizer.adapt(cleaned_X.values)

In [ ]:
len(vectorizer.get_vocabulary())

4828

In [ ]:
vectorized_text=vectorizer(cleaned_X.values)

In [ ]:
dataset=tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8)

In [ ]:
train_data=(int(len(dataset)*.7))
val_data=int(len(dataset)*.2)
test_data=int(len(dataset)*.1)

In [ ]:
train=dataset.take(train_data)
val=dataset.skip(train_data).take(val_data)
text=dataset.skip(train_data+val_data).take(test_data)

In [ ]:
y.shape

(1000, 12)

#Building_Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Bidirectional,Dense,Embedding

In [ ]:
model=Sequential()
model.add(Embedding(Max_features+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(12,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='BinaryCrossentropy')

In [ ]:
hist=model.fit(train,epochs=5,validation_data=val)

Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step - loss: 0.0200 - val_loss: 0.0142
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 112ms/step - loss: 0.0114 - val_loss: 0.0105
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - loss: 0.0076 - val_loss: 0.0131
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - loss: 0.0057 - val_loss: 0.0080
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - loss: 0.0052 - val_loss: 0.0056


#Evaluating the model

In [ ]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy

In [ ]:
pre=Precision()
re=Recall()
acc=CategoricalAccuracy()

In [ ]:
for batch in text.as_numpy_iterator():
  X_true,y_true=batch
  yhat=model.predict(X_true)

  y_true=y_true.flatten()
  yhat=yhat.flatten()

  pre.update_state(y_true,yhat)
  re.update_state(y_true,yhat)
  acc.update_state(y_true,yhat)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


In [ ]:
pre.result().numpy()

np.float32(1.0)

In [ ]:
acc.result().numpy()

np.float32(0.33333334)

In [ ]:
re.result().numpy()

np.float32(1.0)

In [ ]:
model.save("lstm_sentiment_model.h5")

In [ ]:
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

In [ ]:
texts = [
    "You are such an idiot, nobody likes you.",
    "I will kill u.",
    "you are so good",

]

# Preprocess (vectorize)
vectorized_texts = vectorizer(texts)

# Predict
preds = model.predict(vectorized_texts)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


In [ ]:
preds

array([[9.9999440e-01, 9.9999881e-01, 1.9634718e-09, 9.3895032e-05,
        8.9268041e-01, 3.3365755e-05, 8.3751011e-06, 1.6268537e-09,
        1.0290867e-05, 6.8092219e-11, 3.6848288e-11, 1.9647136e-11],
       [9.9484873e-01, 9.9957556e-01, 8.7708666e-04, 4.3331397e-01,
        9.7375941e-01, 5.5816408e-02, 1.3796825e-02, 2.5452208e-04,
        1.3100702e-01, 9.7266501e-08, 1.3997383e-05, 2.0564784e-07],
       [1.3855935e-04, 8.2392828e-05, 4.1628973e-17, 1.7530593e-09,
        3.3363196e-15, 2.6003129e-14, 6.5554062e-13, 1.2607679e-13,
        2.8623410e-22, 4.4736778e-15, 4.7312713e-18, 7.8324602e-16]],
      dtype=float32)

In [ ]:
LABELS = [
    "Toxic", "Abusive", "Threat", "Provocative",
    "Obscene", "Hate Speech", "Racist", "Nationalist",
    "Sexist", "Homophobic", "Religious Hate", "Radicalism"
]

In [ ]:
for text, pred in zip(texts, preds):
    print(f"\nInput: {text}")
    for label, prob in zip(LABELS, pred):
        print(f"{label}: {prob:.2f}")


Input: You are such an idiot, nobody likes you.
Toxic: 1.00
Abusive: 1.00
Threat: 0.00
Provocative: 0.00
Obscene: 0.89
Hate Speech: 0.00
Racist: 0.00
Nationalist: 0.00
Sexist: 0.00
Homophobic: 0.00
Religious Hate: 0.00
Radicalism: 0.00

Input: I will kill u.
Toxic: 0.99
Abusive: 1.00
Threat: 0.00
Provocative: 0.43
Obscene: 0.97
Hate Speech: 0.06
Racist: 0.01
Nationalist: 0.00
Sexist: 0.13
Homophobic: 0.00
Religious Hate: 0.00
Radicalism: 0.00

Input: you are so good
Toxic: 0.00
Abusive: 0.00
Threat: 0.00
Provocative: 0.00
Obscene: 0.00
Hate Speech: 0.00
Racist: 0.00
Nationalist: 0.00
Sexist: 0.00
Homophobic: 0.00
Religious Hate: 0.00
Radicalism: 0.00
